# <font color = 'pickle'>**HW3 - 15 Points** </font>

1. For Questions (1, 2 and 3) you will submit two files: a) A colab notebook b) A well formatted PDF file.
2. The notebook and pdf files should contain all the output.
3. For Question 4 - submit a pdf OR ppt file.
4. Name the files as follows : FirstName_hw3.ipynb, FirstName_hw3.pdf
5. If the submission requires multiple files name them as follows: FirstName_file1_hw3, FirstName_file2_hw3.


In [40]:
import torch
import torch.nn as nn

# <font color = 'pickle'>**Q1. Compute Gradient using  PyTorch Autograd - 2 Points**
## $f(x,y) = \frac{x + \exp(y)}{\log(x) + (x-y)^3}$
Compute dx and dy at x=3 and y=4

In [41]:
def fxy(x,y):
  num = x + torch.exp(y)
  den = torch.log(x) + (x-y)**3
  return num/den

In [42]:
x = torch.tensor([3.0], requires_grad=True)
y = torch.tensor([4.0], requires_grad=True)

In [43]:
f = fxy(x,y)
f

tensor([584.0868], grad_fn=<DivBackward0>)

In [44]:
f.backward()

In [45]:
print(x.grad, y.grad)

tensor([-19733.3965]) tensor([18322.8477])


# <font color = 'pickle'>**Q2. Regression with autogard (backward() method) 5 points** </font>

<font size = 4,  color ='indianred'> **Redo  question 7 from HW1. Now we will use Pytorch's autograd to calculate the gradients instead of manually calculating gradients.**</font>


Imagine that you're trying to figure out relationship between two variables x and y . You have some idea but you aren't quite sure yet whether the dependence is linear or quadratic. 

Your goal is to use least mean squares regression to identify the coefficients for the following three models :

1. Quadratic model where $\mathrm{y} = b + w_1 \cdot \mathrm{x} + w_2 \cdot \mathrm{x}^2$.
1. Linear model where $\mathrm{y} = b + w_1 \cdot \mathrm{x}$.
1. Linear model with no bias  where $\mathrm{y} = w_1 \cdot \mathrm{x}$.


- You will use batch gradient descent to estimate the model co-efficients. Batch gradient descent uses complete training data at each iteration. 
- We will implement only training loop (no splitting of data in to training/validation).
- The training loop will have only one for loop. We need to iterate over whole data in each epoch. We do not need to create batches.
- You may have to try different values of number of epochs/ learning rate to get good results.
- <font color = 'indianred'>**You are not allowed to use Pytorch's nn.module or functions from Pytorch. You will write function for loss function (mean sqaured error), and prediction from scratch.**</font>
- <font color = 'indianred'>**You will not calculate gradients manually. You will use backward() method to compute gradients.**

## <font color = 'pickle'> **Data**

In [46]:
import torch

In [47]:
x = torch.tensor([1.5420291, 1.8935232, 2.1603365, 2.5381863, 2.893443, \
                    3.838855, 3.925425, 4.2233696, 4.235571, 4.273397, \
                    4.9332876, 6.4704757, 6.517571, 6.87826, 7.0009003, \
                    7.035741, 7.278681, 7.7561755, 9.121138, 9.728281])
y = torch.tensor([63.802246, 80.036026, 91.4903, 108.28776, 122.781975, \
                    161.36314, 166.50816, 176.16772, 180.29395, 179.09758, \
                    206.21027, 272.71857, 272.24033, 289.54745, 293.8488, \
                    295.2281, 306.62274, 327.93243, 383.16296, 408.65967])

In [48]:
y = y.view(-1, 1)
x = x.view(-1, 1)
x2 = x*x

In [49]:
x_combined = torch.cat((x, x2), dim = 1)

In [50]:
print(x_combined.shape, x.shape)

torch.Size([20, 2]) torch.Size([20, 1])


## <font color = 'pickle'> **Utility Functions**

### <font color = 'pickle'>**Model Linear Regression**

In [51]:
def linear_reg(x, w, b, bias):
  if bias: 
    return torch.mm(x, w.T) + b
  else: 
    return torch.mm(x, w.T)

### <font color = 'pickle'>**Loss Function**

In [52]:
def mse_loss(y, yhat):
  error = yhat - y
  square_loss = torch.mm(error.T, error)
  return square_loss/len(y)

### <font color = 'pickle'> **Gradient Linear Regression**

In [ ]:
'''
def lin_grad(x, y, y_hat, bias= True):
  error = y_hat - y
  print(len(y))
  w_grad = ( torch.mm(x.T, error))/ len(y) # shape is (x.shape[-1], n_outs) 
  if bias:
    b_grad =  error.sum() /len(y)
    return (w_grad.T, b_grad)
  else:
    return w_grad.T  # reshape to  (n_outs, x.shape[-1]) 
  '''

### <font color = 'pickle'> **Optimizer**

In [54]:
def sgd_step(params, learning_rate):
  with torch.no_grad():
    for param in params:
      param.data -= learning_rate * param.grad    
      param.grad.zero_() 

In [55]:
'''
def sgd_step(params, param_grads, learning_rate):
  for param, param_grad in zip(params, param_grads):
    param -= learning_rate*param_grad
'''    

'\ndef sgd_step(params, param_grads, learning_rate):\n  for param, param_grad in zip(params, param_grads):\n    param -= learning_rate*param_grad\n'

### <font color = 'pickle'> **Train Function**

In [56]:
def train(epochs, x, y, n_outs, bias, loss_function, log_interval, learning_rate):
  loss_epoch = []
  n_ins = x.shape[-1]
  w = torch.normal(0, 0.01, size=(n_outs, n_ins), requires_grad=True)
  b = torch.zeros(n_outs, requires_grad=True)
  
  if bias ==True:
    params = [w,b]
  else:
    params = [w]


  for epoch in range(epochs):
    
    # Step1: forward pass
    y_hat = linear_reg(x, w, b, bias)

    # Step2 : Loss
    loss = loss_function(y_hat, y)

    # Calculate Gradients
    # param_grads = lin_grad(x, y, y_hat, bias)
    loss.backward()
    
    # update parameters
    # sgd_step(params, param_grads, learning_rate) 
    sgd_step(params, learning_rate) 

    if(epoch % log_interval ==0):
      print(f'epoch: {epoch + 1} --> loss {loss.item()}')

  return (w, b) 
 

## <font color = 'pickle'> **Part 1**

In [57]:
# model 1  
loss_function = mse_loss
LEARNING_RATE = 0.0005
EPOCHS = 100000
LOG_INTERVAL= 10000
N_OUTS = 1
BIAS = True

w1, b1 = train(EPOCHS, x_combined, y,  N_OUTS, BIAS, loss_function, LOG_INTERVAL, LEARNING_RATE)

epoch: 1 --> loss 58004.09375
epoch: 10001 --> loss 5.000570774078369
epoch: 20001 --> loss 3.0938141345977783
epoch: 30001 --> loss 2.1370463371276855
epoch: 40001 --> loss 1.6568940877914429
epoch: 50001 --> loss 1.4159462451934814
epoch: 60001 --> loss 1.2948728799819946
epoch: 70001 --> loss 1.2340748310089111
epoch: 80001 --> loss 1.2035744190216064
epoch: 90001 --> loss 1.1881871223449707


In [58]:
print(f' Weights {w1}, \nBias: {b1}')

 Weights tensor([[4.1796e+01, 1.4825e-02]], requires_grad=True), 
Bias: tensor([0.9772], requires_grad=True)


## <font color = 'pickle'> **Part 2**

In [59]:
# model 2
loss_function = mse_loss
LEARNING_RATE = 0.01
EPOCHS = 1000
LOG_INTERVAL= 100
N_OUTS = 1
BIAS = True

w2, b2 = train(EPOCHS, x, y,  N_OUTS, BIAS, loss_function, LOG_INTERVAL, LEARNING_RATE)

epoch: 1 --> loss 57934.76953125
epoch: 101 --> loss 4.354974269866943
epoch: 201 --> loss 2.806662082672119
epoch: 301 --> loss 2.01165509223938
epoch: 401 --> loss 1.6034339666366577
epoch: 501 --> loss 1.3938376903533936
epoch: 601 --> loss 1.2862128019332886
epoch: 701 --> loss 1.2309529781341553
epoch: 801 --> loss 1.2025809288024902
epoch: 901 --> loss 1.1880155801773071


In [60]:
print(f' Weights {w2}, \nBias: {b2}')

 Weights tensor([[41.9377]], requires_grad=True), 
Bias: tensor([0.7467], requires_grad=True)


## <font color = 'pickle'> **Part 3**

In [61]:
# model 2
loss_function = mse_loss
LEARNING_RATE = 0.01
EPOCHS = 10
LOG_INTERVAL= 1
N_OUTS = 1
BIAS = False

w3, b3 = train(EPOCHS, x, y,  N_OUTS, BIAS, loss_function, LOG_INTERVAL, LEARNING_RATE)

epoch: 1 --> loss 57980.26171875
epoch: 2 --> loss 6897.80859375
epoch: 3 --> loss 821.5675659179688
epoch: 4 --> loss 98.80074310302734
epoch: 5 --> loss 12.828018188476562
epoch: 6 --> loss 2.6015257835388184
epoch: 7 --> loss 1.385072946548462
epoch: 8 --> loss 1.2403770685195923
epoch: 9 --> loss 1.223166584968567
epoch: 10 --> loss 1.2211220264434814


In [62]:
print(f' Weights {w3}, \nBias: {b3}')

 Weights tensor([[42.0557]], requires_grad=True), 
Bias: tensor([0.], requires_grad=True)


# <font color = 'pickle'>**Q 3. Numerical Precision - 3 Points**

Given scalars `x` and `y`, implement the following `log_exp` function such that it returns 
$$-\log\left(\frac{e^x}{e^x+e^y}\right)$$.

In [63]:
#Question
def log_exp(x, y):
    ## add your solution here and remove pass
    return -torch.log(torch.exp(x)/(torch.exp(x)+ torch.exp(y)))
    #pass

Test your codes with normal inputs:

In [64]:
x, y = torch.tensor([2.0]), torch.tensor([3.0])
z = log_exp(x, y)
z

tensor([1.3133])

Now implement a function to compute $\partial z/\partial x$ and $\partial z/\partial y$ with `autograd`

In [65]:
def grad(forward_func, x, y): 
  x.requires_grad_()
  y.requires_grad_()
  
  ## Add your codes here
  z=forward_func(x,y)
  z.backward()

  ## your code ends here
  print('x.grad =', x.grad)
  print('y.grad =', y.grad)
  x.grad.zero_()
  y.grad.zero_()



Test your codes, it should print the results nicely. 

In [66]:
grad(log_exp, x, y)

x.grad = tensor([-0.7311])
y.grad = tensor([0.7311])


But now let's try some "hard" inputs

In [67]:
x, y = torch.tensor([50.0]), torch.tensor([100.0])

In [68]:
grad(log_exp, x, y)

x.grad = tensor([nan])
y.grad = tensor([nan])


In [69]:
torch.exp(torch.tensor([100.0]))

tensor([inf])

Does your code return correct results? If not, try to understand the reason. (Hint, evaluate `exp(100)`). Now develop a new function `stable_log_exp` that is identical to `log_exp` in math, but returns a more numerical stable result.
<br> Hint: (1) $\log\left(\frac{x}{y}\right) = log ({x}) -log({y})$
<br> Hint: (2) See logsum Trick - https://www.xarg.org/2016/06/the-log-sum-exp-trick-in-machine-learning/

In [70]:
def stable_log_exp(x, y):
    ## Add your codes here)
    z=torch.max(x,y)
    return -x + z + torch.log(torch.exp(x-z)+ torch.exp(y-z))

In [71]:
log_exp(x, y)

tensor([inf], grad_fn=<NegBackward0>)

In [72]:
stable_log_exp(x, y)

tensor([50.], grad_fn=<AddBackward0>)

In [73]:
grad(stable_log_exp, x, y)

x.grad = tensor([-1.])
y.grad = tensor([1.])


In [5]:
import torch

In [15]:
round(1.1387-0.99,4)

0.1487

In [11]:
a = torch.tensor([0.29])
torch.sigmoid(a)

tensor([0.5720])

# <font color = 'pickle'>**Q4 : Manual Backpropogation (5 Points)**

For the network below update the weights using back propogation. 

<img src = "https://drive.google.com/uc?export=view&id=1e1EI-N773mUtQJ0SQx5wlJJ1RaIQ4dZ2" width =600 >

- X1, x2 are inputs. The values of the inputs are provided.
- h1, h2 are hidden neurons. You will need to calculate the values of h1 and h2 in forward pass.
- o1 and o2 are outputs. 0.01 and 0.99 are the true values of the output. You will calculate the predicted values of o1 and o1 in forward pass.
- W1-W8 are weights. The initial values are provided to you. You will need to calculate the updated values in backward pass.
- bh1, bh2, bo1, bo2 are bias terms. The initial values are provided. You will need to calculate the updated values in backward pass.

- You will apply sigmoid activation on hidden layer.
- You will apply Linear activation function on output neurons.
- You will use the  squared error as the loss function. 
where <br>$ E_1 =1/2 *(\hat{o_1}-o_1)^2 $ <br> 
$ E_2 =1/2 *(\hat{o_2}-o_2)^2 $ <br>  $ E = E_1 + E_2$

Here $E$ is the total loss. $\hat{o_2}$ and $\hat{o_2}$ are predicted values of $o_1$ and $o_2$.
 
- <font color ='indianred'> **Assume a Learning Rate of 10.**
    

<font size = 4 color ='indianred'> **Requirements**
- Show caluclations for one forward and one backward pass.
- Show all the steps of your calculations. You will get partial credit for the steps even if the final answers are not accurate.
- You will do this question manually.
- For this question you can submit - ppt  or pdf file (pdf of handwritten calculations).

##  <font color = 'pickle'>**Q4 : Manual Backpropogation Solution**


###  <font color = 'pickle'>**Forward Pass**

<img src = "https://drive.google.com/uc?export=view&id=1k4GhDnulYPrOueTtAYEUeVrRWG4PMEjw" width =600, height = 350 >



In [4]:
round(0.5 - 10*0.1486*0.562,4)

-0.3351

###  <font color = 'pickle'>**Backward Pass**

<img src = "https://drive.google.com/uc?export=view&id=1k5AE6KeOfYeXYo6x_RoLUEiUFi6dlt4w" width =600, height = 350 >



## <font color = 'pickle'>**Q4 : Manual Backpropogation Solution Using PyTorch**

In [74]:
x= torch.tensor([[0.0,0.3]]).float()

In [75]:
x.shape

torch.Size([1, 2])

In [76]:
y= torch.tensor([[0.01, 0.99]]).float()

In [77]:
y.shape

torch.Size([1, 2])

In [78]:
model1  = nn.Sequential(nn.Linear(2,2), nn.Sigmoid(), nn.Linear(2,2))

In [79]:
model1[0].weight = torch.nn.Parameter(torch.tensor([[0.10, 0.50],[0.4, 0.30]]))
model1[0].bias = torch.nn.Parameter(torch.tensor([[0.1, 0.2]]))
model1[2].weight = torch.nn.Parameter(torch.tensor([[0.2, 0.60], [0.5, 0.8]]))
model1[2].bias = torch.nn.Parameter(torch.tensor([[0.3, 0.4]]))

In [80]:
predy = model1(x)

In [81]:
predy

tensor([[0.7556, 1.1387]], grad_fn=<AddmmBackward0>)

In [82]:
for name, parameter in model1.named_parameters():
  print(name, parameter)

0.weight Parameter containing:
tensor([[0.1000, 0.5000],
        [0.4000, 0.3000]], requires_grad=True)
0.bias Parameter containing:
tensor([[0.1000, 0.2000]], requires_grad=True)
2.weight Parameter containing:
tensor([[0.2000, 0.6000],
        [0.5000, 0.8000]], requires_grad=True)
2.bias Parameter containing:
tensor([[0.3000, 0.4000]], requires_grad=True)


In [83]:
optim = torch.optim.SGD(model1.parameters(), lr=10)

In [84]:
criterion = nn.MSELoss(reduction='mean')

In [85]:
loss = criterion(predy, y)

In [86]:
loss

tensor(0.2890, grad_fn=<MseLossBackward0>)

In [87]:
e=predy-y
e

tensor([[0.7456, 0.1487]], grad_fn=<SubBackward0>)

In [88]:
0.5*(e*e).sum()

tensor(0.2890, grad_fn=<MulBackward0>)

In [89]:
loss.backward()

In [90]:
optim.step()

In [91]:
for name, parameter in model1.named_parameters():
  print(name, parameter)

0.weight Parameter containing:
tensor([[ 0.1000,  0.3350],
        [ 0.4000, -0.1159]], requires_grad=True)
0.bias Parameter containing:
tensor([[-0.4500, -1.1865]], requires_grad=True)
2.weight Parameter containing:
tensor([[-3.9918, -3.6650],
        [-0.3359, -0.0505]], requires_grad=True)
2.bias Parameter containing:
tensor([[-7.1563, -1.0869]], requires_grad=True)
